# 🏗️ 3. Warehouse Modeling with SCD
This notebook builds a star schema and adds SCD logic to dimension tables.

## 📐 Create Dimensions and Fact Table

In [ ]:
sales_clean = spark.read.parquet("warehouse/sales_logs")
sales_clean.createOrReplaceTempView("sales_logs")

dim_customer = sales_clean.select("customer_id").distinct()
dim_product = sales_clean.select("product_id").distinct()
dim_region = sales_clean.select("region").distinct()

## 📅 Create dim_time Table

In [ ]:
from pyspark.sql.functions import year, month, dayofmonth, dayofweek, date_format
dim_time = sales_clean.select("timestamp").withColumn("date", col("timestamp").cast("date"))     .withColumn("year", year("timestamp"))     .withColumn("month", month("timestamp"))     .withColumn("day", dayofmonth("timestamp"))     .withColumn("weekday", dayofweek("timestamp"))     .withColumn("month_name", date_format("timestamp", "MMMM")).dropDuplicates(["date"])

## 🧾 Create fact_sales and Enrich with SCD2

In [ ]:
# Placeholder for SCD2 implementation from earlier notebook